In [3]:
import great_expectations as gx
import pandas as pd
import warnings

# Stäng specifika varningar från GX
warnings.filterwarnings("ignore", message="`result_format` configured at the Validator-level*")

# 1. Läs in data
df = pd.read_csv("data/clean_transactions.csv")
df.index += 2
df["timestamp"] = pd.to_datetime(df["timestamp"])

# 2. Skapa Great Expectations-kontext
context = gx.get_context()

# 3. Lägg till pandas-datakälla
data_source = context.data_sources.add_pandas(name="pandas")
data_asset = data_source.add_dataframe_asset(name="transactions_data")
batch_def = data_asset.add_batch_definition_whole_dataframe(name="batch_def")
batch = batch_def.get_batch(batch_parameters={"dataframe": df})

# 4. Skapa förväntningssuite
suite = gx.core.expectation_suite.ExpectationSuite(name="transactions_suite")
validator = context.get_validator(batch=batch, expectation_suite=suite)

# 5. Funktionsbaserade valideringar
def validate_transaction_core(v):
    v.expect_column_values_to_not_be_null("transaction_id")
    v.expect_column_values_to_be_unique("transaction_id")
    v.expect_column_values_to_not_be_null("timestamp")
    v.expect_column_values_to_be_between("amount", min_value=0.01, max_value=100000)

def validate_currency(v):
    v.expect_column_values_to_not_be_null("currency")
    v.expect_column_values_to_match_regex("currency", r"^[A-Z]{3}$")

def validate_accounts(v):
    v.expect_column_values_to_not_be_null("sender_account")
    v.expect_column_values_to_not_be_null("receiver_account")

def validate_location(v):
    v.expect_column_values_to_not_be_null("sender_country")
    v.expect_column_values_to_not_be_null("receiver_country")
    v.expect_column_values_to_not_be_null("sender_municipality")
    v.expect_column_values_to_not_be_null("receiver_municipality")

def validate_other(v):
    v.expect_column_values_to_not_be_null("transaction_type")
    v.expect_column_values_to_not_be_null("notes")

# 6. Kör alla valideringar
validate_transaction_core(validator)
validate_currency(validator)
validate_accounts(validator)
validate_location(validator)
validate_other(validator)

# 7. Kör validering en gång
results = validator.validate()

# 8. Skriv ut resultat per kolumn
for result in results["results"]:
    config = result.expectation_config.to_json_dict()
    result_data = result.result

    column = config.get("kwargs", {}).get("column", "(Table)")
    expectation = config.get("expectation_type", "Unknown expectation")
    success = result.success

    print(f"{column} → {expectation}: {'None!' if success else 'Detected'}")

    if not success:
        unexpected_count = result_data.get("unexpected_count", "unknown")
        unexpected_values = result_data.get("partial_unexpected_list", [])

        print(f"Number of unexpected values: {unexpected_count}")
        if unexpected_values:
            print(f"Unexpected values: {unexpected_values}")
        print()


Calculating Metrics: 100%|██████████| 41/41 [00:00<00:00, 138.89it/s]

transaction_id → Unknown expectation: None!
transaction_id → Unknown expectation: None!
timestamp → Unknown expectation: None!
amount → Unknown expectation: None!
currency → Unknown expectation: None!
currency → Unknown expectation: None!
sender_account → Unknown expectation: None!
receiver_account → Unknown expectation: None!
sender_country → Unknown expectation: None!
receiver_country → Unknown expectation: None!
transaction_type → Unknown expectation: None!
notes → Unknown expectation: Detected
Number of unexpected values: 9863
Unexpected values: [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]

